In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

/home/jo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [28]:
from SCFInitialGuess.utilities.dataset import Dataset

S, P = np.load("butadien/dataset.npy")
dim = 26
dataset = Dataset(S, P)

[-] 2018-03-20 11:48:56: Data set initialized. Mean value std: 0.00872792525661018


## Training different costs

In [35]:
from SCFInitialGuess.nn.networks import EluTrNNN
from SCFInitialGuess.nn.training import Trainer, MSE

class SymmetryPenalty(MSE):
    def __init__(self, coupling=1e-4):
        """alpha: coupling parameter"""
        
        self.coupling = coupling
        
    def function(self, network, y_placeholder):

        error = \
            super(SymmetryPenalty, self).function(network, y_placeholder)

        penalty = self.symmetry_measure(network.output_tensor) * self.coupling

        cost = error + penalty

        tf.summary.scalar("symmetry_penalty", penalty)
        tf.summary.scalar("total_loss", cost)

        return cost
    
    def symmetry_measure(self, output_tensor):
        
        reshaped = tf.reshape(output_tensor, [-1, dim, dim])
        transposed = tf.matrix_transpose(reshaped)
        return tf.norm(reshaped - transposed, axis=(1,2))

    
trainer = Trainer(
    EluTrNNN([dim**2, 200, 100, dim**2]),
    cost_function=SymmetryPenalty(coupling=1e-3)
)

trainer.setup()
network, sess = trainer.train(dataset)
        
        

[-] 2018-03-20 11:51:20: No target graph specified for Trainer setup. Creating new graph ...
[-] 2018-03-20 11:51:20: Setting up the training in the target graph ...
[-] 2018-03-20 11:51:20: network ...
[-] 2018-03-20 11:51:20: error function ...
[-] 2018-03-20 11:51:20: cost function ...
[-] 2018-03-20 11:51:21: training step


ValueError: Fetch argument <tf.Operation 'init' type=NoOp> cannot be interpreted as a Tensor. (Operation name: "init"
op: "NoOp"
 is not an element of this graph.)

## Analysis

In [26]:
tf.reset_default_graph()

A = np.array([np.ones(4).reshape(2, 2), np.arange(4).reshape(2, 2)])

with tf.Session() as sess:
    a = tf.constant(A)
    t = tf.matrix_transpose(a)
    diff = a-t
    norm = tf.norm(diff, axis=(1,2))
    
    print(
        sess.run(norm)
    )

[0.         1.41421356]
